# helper_utilities.ipynb
> Helper functions for when we need to work with files in Google Colab or locally

In this notebook, we write some generic code to help us interface more easily with loading in files.

In [ ]:
#| default_exp IOHelperUtilities

In [ ]:
#| export
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial
from ipyfilechooser import FileChooser
import os

In [ ]:
#| export
def check_is_colab():
    """
    Check if the current environment is Google Colab.
    """
    try:
        import google.colab
        return True
    except:
        return False

In [ ]:
assert not check_is_colab(), 'On this system, we should not be in Colab'

## File Choosers
Jupyter notebooks, the different IDEs, and ipywidgets currently (as of August 8, 2023) are not playing nice together, and also are misaligned in terms of versions. What works in Jupyter Lab at version 8 somehow doesn't work in Google Colab and changes are needed. Neither the Google Colab version or the Jupyter Lab version work with VSCode.

While this is being worked out between VS Code developers and ipywidgets, we've found a mid-term solutions which requires another package. We implement and test this below (thanks, Code Interpreter!)

In [ ]:
#| export
class MultiFileChooser:
    def __init__(self):
        self.fc = FileChooser('.')
        self.fc.title = "Use the following file chooser to add each file individually.\n You can remove files by clicking the remove button."
        self.fc.use_dir_icons = True
        self.fc.show_only_dirs = False
        self.selected_files = []
        
        self.fc.register_callback(self.file_selected)
        
        self.output = widgets.Output()
        
    def file_selected(self, chooser):
        if self.fc.selected is not None and self.fc.selected not in self.selected_files:
            self.selected_files.append(self.fc.selected)
            self.update_display()
    
    def update_display(self):
        with self.output:
            clear_output()
            for this_file in self.selected_files:
                remove_button = widgets.Button(description="Remove", tooltip="Remove this file")
                remove_button.on_click(partial(self.remove_file, file=this_file))
                display(widgets.HBox([widgets.Label(value=this_file), remove_button]))
    
    def remove_file(self, button, this_file):
        if this_file in self.selected_files:
            self.selected_files.remove(this_file)
            self.update_display()
    
    def display(self):
        display(self.fc, self.output)
    
    def get_selected_files(self):
        return self.selected_files

Now we test the file chooser very briefly to ensure that the results are as we desire.

In [ ]:
# Create file chooser and interact
mfc = MultiFileChooser()
mfc.display()

FileChooser(path='/workspaces/lo-achievement/nbs', filename='', title='Use the following file chooser to add e…

Output()

In [ ]:
# get files that were selected.
mfc.get_selected_files()

['/workspaces/lo-achievement/nbs/_quarto.yml',
 '/workspaces/lo-achievement/nbs/nbdev.yml']

## File loading
Now, we implement a file chooser that will work across platforms, whether it be Google Colab or local environments.

In [ ]:
#| export
def setup_drives(upload_set):

    upload_set = upload_set.lower()
    uploaded = None

    # allow them to mount the drive if they chose Google Colab.
    if upload_set == 'google drive':
        if check_is_colab():
            from google.colab import drive
            drive.mount('/content/drive')
        else:
            raise ValueError("It looks like you're not on Google Colab. Google Drive mounting is currently only implemented for Google Colab.")

    # Everything else means that they'll need to use a file chooser (including Google Drive)
    if check_is_colab():
        from google.colab import files
        uploaded = files.upload()
    else:
        # Create file chooser and interact
        mfc = MultiFileChooser()
        mfc.display()
        uploaded = mfc.get_selected_files()
    
    return uploaded

In [ ]:
res = setup_drives('local drive')

FileChooser(path='/workspaces/lo-achievement/nbs', filename='', title='Use the following file chooser to add e…

Output()

In [ ]:
res

['/workspaces/lo-achievement/nbs/_quarto.yml',
 '/workspaces/lo-achievement/nbs/nbdev.yml']

Now, we'll verify the behavior of Google Drive. We'll wrap this in a try/except block so the code can run all the way through.

In [ ]:
try:
    setup_drives('google drive')
except Exception as e:
    print(f"An exception of type {type(e).__name__} occurred. Arguments:\n{e}")

An exception of type ValueError occurred. Arguments:
It looks like you're not on Google Colab. Google Drive mounting is currently only implemented for Google Colab.


## Future expected implementation

The following code is included as it works, just not in Visual Studio code. The current implementation of the File chooser is a bit inelegant, but this is due to the current limitations of the combination of the libraries and platforms. Once some errors with VS code can be updated, this code will be the preferable solution as it is more familiar to users.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

class UniversalFileUpload:

    def __init__(self):
        self.filelist = []
        self.uploader = None
        self.status_output = None
    
    def _process_upload(self, change):
        self.status_output.clear_output()
        with self.status_output:
            print('What is happening?')
            print(change)

    def process_uploads(self, change):
        if change['new'] and change['new'] != None:
          with self.status_output:
              print(change)
          
          self.filelist = change['new']
          
          #get filenames and promt
          fnames = [fileinfo['name'] for fileinfo in self.filelist['metadata']]
          with self.status_output:
              print('Uploaded files:', fnames)
          
          #clear it so it doesn't save state
          self.uploader.close()
    
    def get_upload_value(self):
        return self.filelist
    
    def choose_files(self):
        self.uploader = widgets.FileUpload(accept='', multiple=True, description='cat')
        self.status_output = widgets.Output()
        self.file_output_box = widgets.VBox([self.uploader, self.status_output])
        self.uploader.observe(self._process_upload)

        with self.status_output:
            print('Waiting...')

        return self.file_output_box

In [ ]:
#test
ul = UniversalFileUpload()
ul.choose_files()